In [1]:
# audio data fed into normalized LSTM with Conv layer


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [7]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Conv1D, Dropout
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [3]:
# uploading the X values
#X_input = load_Xinput(get_fc6_directory(7)) #VGG16 data

# X_input = np.load('../Desktop/InceptionResNetV2_feature_data.npy') #resnet data!

X_input = np.load('../Desktop/audio_feature_data.npy')

print(X_input.shape)

(7, 207, 1583)


In [4]:
# uploading the Y values
y_data_input = np.load('../Desktop/labels.npy')
print(y_data_input.shape)

(7, 219)


In [5]:
timesteps = X_input.shape[1]
data_dim = X_input.shape[2]

X_train = X_input[:4,:timesteps, :]
Y_train = y_data_input[:4, :timesteps]

X_test = X_input[4:, :timesteps, :]
Y_test = y_data_input[4:, :timesteps]

print(X_test.shape, X_train.shape, Y_train.shape, Y_test.shape)

(3, 207, 1583) (4, 207, 1583) (4, 207) (3, 207)


In [8]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()

# normalization
model.add(BatchNormalization(input_shape=(timesteps, data_dim)))

model.add(Conv1D(10, kernel_size=3, activation="relu"))

model.add(LSTM(timesteps, return_sequences=True))
#input_shape=(timesteps, data_dim)

model.add(Dropout(0.5))

model.add(LSTM(timesteps, return_sequences=True))

# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works
# add the final dense layer and then softmax
model.add(Dense(timesteps, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [9]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [10]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [11]:
# running the LSTM model
model.fit(X_train, Y_train, epochs = 20, batch_size = 128, validation_data=(X_test, Y_test))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 3s 747ms/step - loss: 0.6948 - binary_accuracy: 0.4734 - FScore2: 0.2891 - val_loss: 1.3670 - val_binary_accuracy: 0.5507 - val_FScore2: 0.0885
Epoch 2/20
4/4 [==============================] - 1s 286ms/step - loss: 0.9369 - binary_accuracy: 0.6582 - FScore2: 0.2066 - val_loss: 4.4408 - val_binary_accuracy: 0.6248 - val_FScore2: 0.2922
Epoch 3/20
4/4 [==============================] - 1s 344ms/step - loss: 2.1960 - binary_accuracy: 0.8007 - FScore2: 0.5338 - val_loss: 2.2507 - val_binary_accuracy: 0.5233 - val_FScore2: 0.4208
Epoch 4/20
4/4 [==============================] - 1s 233ms/step - loss: 1.6123 - binary_accuracy: 0.5314 - FScore2: 0.3648 - val_loss: 3.6917 - val_binary_accuracy: 0.7021 - val_FScore2: 0.0204
Epoch 5/20
4/4 [==============================] - 1s 230ms/step - loss: 1.1203 - binary_accuracy: 0.8490 - FScore2: 0.4444 - val_loss: 4.1692 - val_binary_accuracy: 0.4589 - val_FSco

In [17]:
model.save('./audio_cnn_norm_drop_lstm')

TypeError: save() got an unexpected keyword argument 'include_metric'

In [13]:
# trying to view the model output
out = model.predict(X_train)

movie_index = 2

print("model prediction:")
print(out[movie_index])
print("target:")
print(Y_train[movie_index])

print("before 64:")
print(out[movie_index][63])
print("64:")
print(out[movie_index][65])

print("rounded")
print(np.round(out)[movie_index])

model prediction:
[8.73011188e-04 6.73524220e-04 7.57076486e-04 1.19396381e-03
 9.97976866e-04 6.59375102e-04 8.29332275e-04 6.85389154e-04
 1.01351645e-03 1.10705744e-03 8.16348649e-04 1.13546080e-03
 8.59474472e-04 1.04014564e-03 8.94202676e-04 7.48174440e-04
 1.21196196e-03 8.05895485e-04 1.22030987e-03 8.38304055e-04
 1.15930708e-03 8.40465538e-04 1.07808376e-03 9.90873203e-04
 1.19074387e-03 1.12607400e-03 4.87624377e-01 5.11097908e-01
 5.19720852e-01 4.96523529e-01 5.17930984e-01 4.92724568e-01
 5.02635181e-01 5.02999961e-01 5.19610763e-01 5.00069678e-01
 5.01542687e-01 8.48218679e-01 8.41657639e-01 4.88829970e-01
 4.96384263e-01 4.82674688e-01 5.11160612e-01 5.14417768e-01
 5.10625303e-01 5.15987575e-01 5.10102928e-01 5.04541218e-01
 5.08259594e-01 5.08650064e-01 5.13549626e-01 5.11653006e-01
 8.83008957e-01 8.33679795e-01 8.62520516e-01 8.65613878e-01
 8.61397564e-01 8.76735866e-01 8.73146296e-01 8.35579157e-01
 4.97643769e-01 5.01167357e-01 5.00925422e-01 5.08317232e-01
 2.034

In [14]:
#try visualizing this model at some point?